Traduction du texte avec Translators et google.

In [1]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer

import RktnChallenge.preprocessing.translater
importlib.reload(RktnChallenge.preprocessing.translater)
from RktnChallenge.preprocessing.translater import LanguageDetector
from RktnChallenge.preprocessing.translater import Translator

import RktnChallenge.preprocessing.CopyWhereNan
importlib.reload(RktnChallenge.preprocessing.CopyWhereNan)
from RktnChallenge.preprocessing.CopyWhereNan import CopyWhereNan

import RktnChallenge.preprocessing.removeHTML
importlib.reload(RktnChallenge.preprocessing.removeHTML)
from RktnChallenge.preprocessing.removeHTML import removeHTML

2023-09-17 19:35:01.166731: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-17 19:35:01.192841: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9511] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-17 19:35:01.192906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-17 19:35:01.192979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-17 19:35:01.198419: I tensorflow/core/platform/cpu_feature_guar

On charge la base et init le support GPU / CPU

In [3]:
RktnModel = ModelTrainer("data.csv")

# On initialise le GPU disponible, la gestion mémoire
#RktnModel.initGPU()

On détecte la langue de la source et range dans une nouvelle colonne

In [4]:
desc_removeHTML = removeHTML("description")
desi_removeHTML = removeHTML("designation")

desi_langueDetect = LanguageDetector("desi_langue","designation",confiance=0.8)
desc_langueDetect = LanguageDetector("desc_langue","description",confiance=0.8)


desi_Translator = Translator(dest = "tr_designation", src="designation",               
                             detected_lang="desi_langue", source = "google",verbose = True)

desc_Translator = Translator(dest = "tr_description", src="description",               
                             detected_lang="desc_langue", source = "google", verbose = True)



RktnModel.initPreprocess()
# unidecode et removeHTML
RktnModel.addPreprocessStep("desc removeHTML",desc_removeHTML)
RktnModel.addPreprocessStep("desi removeHTML",desi_removeHTML)
# decode la langue et traduit la designation
RktnModel.addPreprocessStep("detect langue designation",desi_langueDetect)
RktnModel.addPreprocessStep("translate designation",desi_Translator)
# decode la langue et traduit la description
RktnModel.addPreprocessStep("detect langue description",desc_langueDetect)
RktnModel.addPreprocessStep("translate description",desc_Translator)
RktnModel.preprocess()


print("langues detectees dans les designations: ",desi_langueDetect.langues)
print("langues detectees dans les descriptions: ",desc_langueDetect.langues)



4pcs Ensemble en acier inoxydable haut de gamme Dingers de terre couverts de couverts de fourchette de fourchette de fourchette Points de vente de produits: le couverte est idéale pour la maison quotidienne, HotelrestaurantPartyBuffetBistrocampingPicNicetc. avec la boîte à cadeau de fenêtre de luxe. durable pendant des années. Acier inoxydable de haute qualité et anti-corrosion; L'anti-oxydation peut la vaisselle à long terme de la longueur de longueur de longueur de longueur est basée sur un toucher humain réel pour créer un palmier pratiquement et confortable à utiliser la forme parabole élégante de la forme du corps met en évidence les matières premières de haute qualité. Les couleurs de l'effet peuvent avoir une différence légèrement. Veuillez permettre des erreurs de 1 à 2 cm en raison de la mesure manuelle. Merci pour votre compréhension de compréhension Incluez: 4pcs Set en acier inoxydable Hostume de vaisselle couverte de couverts de fourchette
Un merveilleux ajout à tout espac

On vient de traduire dans tr_ ce qui n'est pas en francais, on complete avec le français:

In [ ]:
# on copie les lignes qui n'ont pas été traduites (celles en français)
desc_copy = CopyWhereNan("tr_description","description")
desi_copy = CopyWhereNan("tr_designation","designation")

RktnModel.initPreprocess()
RktnModel.addPreprocessStep("desc copy french",desc_copy)
RktnModel.addPreprocessStep("desi copy french",desi_copy)
RktnModel.data = RktnModel.preprocess()


#on sauve
RktnModel.saveCSV("data_tr.csv")
